In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv('/content/merged_2024.csv')

In [3]:
df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,TAIL_NUM,OP_CARRIER_FL_NUM,...,DEP_DELAY_NEW,DEP_TIME_BLK,CRS_ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_TIME_BLK,CANCELLED,CRS_ELAPSED_TIME,DISTANCE,WEATHER_DELAY
0,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,N133EV,5284,...,0.0,1800-1859,2045,-26.0,0.0,2000-2059,0.0,150.0,722.0,NaN
1,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,N133EV,5341,...,0.0,0800-0859,1028,-21.0,0.0,1000-1059,0.0,118.0,641.0,NaN
2,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,N133EV,5390,...,0.0,1400-1459,1621,-20.0,0.0,1600-1659,0.0,116.0,599.0,NaN
3,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,N133EV,5390,...,0.0,1100-1159,1338,-34.0,0.0,1300-1359,0.0,138.0,599.0,NaN
4,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,N134EV,5492,...,4.0,1200-1259,1420,-1.0,0.0,1400-1459,0.0,98.0,374.0,NaN


In [4]:
df.shape

(55236, 31)

In [5]:
df['ORIGIN_STATE_ABR'].unique()

array(['NY', 'SC', 'NC', 'MO', 'MI', 'OH', 'KY', 'VA', 'FL', 'IN', 'ME',
       'RI', 'GA', 'TN', 'AL', 'VT', 'MD', 'IA', 'CA', 'TX', 'AZ', 'CO',
       'IL', 'MA', 'VI', 'OR', 'WA', 'WI', 'LA', 'NV', 'PR', 'UT', 'MN',
       'HI', 'NJ', 'OK', 'PA', 'AR', 'NE', 'MT', 'CT'], dtype=object)

In [6]:
def convert_hhmm_to_time(hhmm):
    if pd.isnull(hhmm) or hhmm == '':
        return None
    hhmm = int(hhmm)
    hours = hhmm // 100
    minutes = hhmm % 100
    # Handle cases where hhmm is invalid
    if hours >= 24 or minutes >= 60:
        return None
    return datetime.strptime(f'{hours:02d}:{minutes:02d}', '%H:%M').time()


In [7]:
def get_departure_datetime(row):
    # Extract FL_DATE
    FL_DATE = row['FL_DATE']
    try:
        # Convert FL_DATE to datetime.date object
        flight_date = pd.to_datetime(FL_DATE).date()
    except Exception as e:
        # Handle parsing errors
        return pd.Series({'departure_datetime': None})

    # Extract CRS_DEP_TIME
    DEP_TIME = row.get('CRS_DEP_TIME')

    # Use scheduled time if actual time is missing
    if pd.isnull(DEP_TIME) or DEP_TIME == '':
        DEP_TIME = row.get('DEP_TIME')

    # Convert time to datetime.time object
    dep_time = convert_hhmm_to_time(DEP_TIME)

    # Return None if time is invalid
    if dep_time is None:
        return pd.Series({'departure_datetime': None})

    # Combine flight_date and dep_time to get departure_datetime
    departure_datetime = datetime.combine(flight_date, dep_time)

    return pd.Series({'departure_datetime': departure_datetime})


In [8]:
# Apply the function to each row in the DataFrame
df[['departure_datetime']] = df.apply(get_departure_datetime, axis=1)

In [9]:
df.to_csv('updated_merged_2024.csv',index=False)

In [10]:
# Load the CSV files
weather_df = pd.read_csv('/content/stations_20240101_20240201_filtered.csv')  # Weather CSV file path
flights_df = pd.read_csv('/content/updated_merged_2024.csv')  # Flights CSV file path

# Drop unwanted columns from the weather data
weather_df.drop(columns=['ice_accretion_1hr', 'ice_accretion_3hr', 'ice_accretion_6hr'], inplace=True)

# Convert the 'valid' column in weather data (24-hour format)
weather_df['valid'] = pd.to_datetime(weather_df['valid'], format='%Y-%m-%d %H:%M')


# Convert the columns to datetime format
flights_df['departure_datetime'] = pd.to_datetime(flights_df['departure_datetime'], format='%Y-%m-%d %H:%M:%S')
# weather_df['valid'] = pd.to_datetime(weather_df['valid'], format='%Y-%m-%d %H:%M:%S')

# Round to the nearest hour (this will automatically handle date increments)
flights_df['rounded_departure'] = flights_df['departure_datetime'].dt.round('H')
weather_df['rounded_valid'] = weather_df['valid'].dt.round('H')


# Perform the left join on the entire flights_df, matching on 'rounded_departure', 'rounded_valid', and 'ORIGIN' with 'station'
merged_df = pd.merge(
    flights_df,
    weather_df,
    left_on=['rounded_departure', 'ORIGIN'],
    right_on=['rounded_valid', 'station'],
    how='left'
)


In [11]:
merged_df.to_csv('final_merged_2024.csv',index=False)